# Self Attention Prototype


In [ ]:
!pip install torch==2.3.0
!pip install torchtext==0.18
# !pip install torchvision==0.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
# install check
import torch, torchtext

print("torch:", torch.__version__)
print("torchtext:", torchtext.__version__)

torch: 2.3.0+cu121
torchtext: 0.18.0+cpu


In [ ]:
# Google Drive Load
from google.colab import drive
drive.mount('/content/drive' )

import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/SKALA/GAI2.LLM')


Mounted at /content/drive


### sonnets 코퍼스의 vocabulary 만들기

In [ ]:
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Read the text file
with open('sonnets.txt', 'r') as file:
    text = file.read()

# Tokenize the text  (using simple tokenizer)
tokenizer = get_tokenizer('basic_english')
tokens = tokenizer(text)     # 전체 텍스트를 토큰 리스트로 변환
unique_tokens = set(tokens)  # 토큰 리스트에서 중복 제거

### unique_tokens 집합을 이용해 단어 → 숫자(stoi) / 숫자 → 단어(itos) 매핑 만들기


In [ ]:
stoi = { s:i for i,s in enumerate(unique_tokens)}
itos = { i:s for i,s in enumerate(unique_tokens)}
print(stoi)
print(itos)

vocab_size = len(unique_tokens)
print(vocab_size)

{'seasons': 0, 'persuade': 1, 'monsters': 2, 'coming': 3, 'place': 4, 'compounds': 5, 'without': 6, 'travel': 7, 'dregs': 8, 'maid': 9, 'user': 10, 'everywhere': 11, 'self-love': 12, 'make': 13, 'breathers': 14, 'of': 15, 'spot': 16, 'prouder': 17, 'reviewest': 18, 'dispense': 19, 'shines': 20, 'dreaming': 21, 'sorry': 22, 'beseem': 23, 'leisure': 24, 'inheritors': 25, 'frost': 26, 'skill': 27, 'better': 28, 'climbed': 29, 'ornaments': 30, 'victors': 31, 'pleased': 32, 'fulness': 33, 'can': 34, 'uphold': 35, 'mud': 36, 'ring': 37, 'painting': 38, 'ink': 39, 'bent': 40, 'fruit': 41, 'lovely': 42, 'damasked': 43, 'strange': 44, 'shore': 45, 'essays': 46, 'frailties': 47, 'vouchsafe': 48, 'chiding': 49, 'perusal': 50, 'seek': 51, 'survive': 52, 'huge': 53, 'shamed': 54, 'hid': 55, 'thanks': 56, 'blunting': 57, 'ruined': 58, 'incapable': 59, 'thence': 60, 'friends': 61, 'knows': 62, 'bears': 63, 'yea': 64, 'lodged': 65, 'other': 66, 'defeat': 67, 'decree': 68, 'complexion': 69, 'inward': 7

### 문장을 정수 인덱스로 바꾸고 → 임베딩 층을 거쳐 벡터 표현으로 변환

In [ ]:
sentence = "i love you"
indices = [stoi[word] for word in sentence.split()]  # 띄어쓰기 기준으로 분리하고 숫자 인덱스 변환
print(indices)

import torch.nn as nn

# 입력: 단어 인덱스 (정수)
# 출력: 각 단어의 임베딩 벡터

embedding_dim = 20
embedding = nn.Embedding(vocab_size, embedding_dim)

embedded_sentence = embedding(torch.tensor(indices))
print(embedded_sentence)


[2485, 119, 2189]
tensor([[-0.7202, -1.4790,  0.0096,  0.2148, -0.6859, -0.0440, -0.4941,  0.4224,
          0.3153,  0.7374,  2.0883,  0.8282,  0.3581, -0.1634,  0.2305, -0.9165,
          1.2477,  0.2477,  1.1511, -0.0563],
        [ 0.3011, -0.1361, -0.6116,  0.3114, -0.1752, -0.8227,  1.4952,  1.3835,
         -1.6882,  0.8879, -0.6785, -1.5725, -0.1478, -0.3587, -0.9277, -0.7567,
          1.2114,  1.3855, -1.1729,  0.2567],
        [ 1.0861,  0.5161,  2.2962,  0.6927, -1.6665,  0.5236,  1.0746,  1.2003,
          0.7955,  0.0320,  0.4892, -2.2618,  1.1577, -0.1324,  0.9927,  0.0076,
         -1.4505,  0.1624, -0.7121,  0.9599]], grad_fn=<EmbeddingBackward0>)


### Self Attention

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SelfAttention(nn.Module):
    def __init__(self, embed_dim, atten_dim):
        super().__init__()
        self.query = nn.Linear(embed_dim, atten_dim, bias=False)
        self.key = nn.Linear(embed_dim, atten_dim, bias=False)
        self.value = nn.Linear(embed_dim, atten_dim, bias=False)

    def forward(self, x):
        query = self.query(x)
        key = self.key(x)
        value = self.value(x)

        scores = torch.matmul(query, key.transpose(-2, -1))
        scores = scores / key.size(-1)**0.5

        attention_weights = F.softmax(scores, dim=-1)
        weighted_values = torch.matmul(attention_weights, value)

        return weighted_values

In [ ]:
# Create an instance of SelfAttention
# 벡터들을 서로 비교해서 관계(유사도)를 계산하고 새로운 표현(예: 5차원)으로 변환
# 출력은 각 단어가 문장 내 다른 단어와의 관계를 반영한 컨텍스트 벡터

atten_dim = 5
self_attention = SelfAttention(embedding_dim, atten_dim)

output = self_attention(embedded_sentence)
print(output)

tensor([[ 0.3025, -0.3198,  0.1056,  0.0735, -0.0726],
        [ 0.0955, -0.1943,  0.1120,  0.0607,  0.0151],
        [-0.1662, -0.0401,  0.1317,  0.0092,  0.1101],
        [-0.0298, -0.1407,  0.1216, -0.0071,  0.0339]], grad_fn=<MmBackward0>)
